# Mini Project-1

In [ ]:
from google.colab import files
upload = files.upload()

Saving titanic.csv to titanic.csv


In [ ]:
import pandas as pd
import numpy as np
import io as io
from sklearn.preprocessing import MinMaxScaler

df=pd.read_csv(io.BytesIO(upload["titanic.csv"])) #original data

df_normalised = df.copy()           
  
# applying normalization techniques on age and fare because their values are large as compared to other features
column = 'Age'    #Normalization age of passengers 
df_normalised[column] = MinMaxScaler().fit_transform(np.array(df_normalised[column]).reshape(-1,1))
column = 'Fare'       #Normalization Fare of passengers 
df_normalised[column] = MinMaxScaler().fit_transform(np.array(df_normalised[column]).reshape(-1,1))

# view normalized data  
df= df_normalised
df


,Survived,(Intercept),Sexmale,Age,SibSp,Parch,Fare
0,0,1,1,0.271174,1,0,0.014151
1,1,1,0,0.472229,1,0,0.139136
2,1,1,0,0.321438,0,0,0.015469
3,1,1,0,0.434531,1,0,0.103644
4,0,1,1,0.434531,0,0,0.015713
...,...,...,...,...,...,...,...
707,0,1,0,0.484795,0,5,0.056848
708,0,1,1,0.334004,0,0,0.025374
709,1,1,0,0.233476,0,0,0.058556
710,1,1,1,0.321438,0,0,0.058556


In [ ]:
# df=pd.read_csv(io.BytesIO(upload["titanic.csv"]))
# df
list_Y= []    # contains survival data 1 or 0
for i in range(len(df)):
  list_Y.append(df['Survived'][i])

df2= df
df2.pop('Survived')
list_X=[]     # contains covariates where list[i] will give ith covariate
for i in range(len(df)):
  list_X.append(df2.iloc[i].to_numpy())


In [ ]:
from scipy.stats import norm             # for cdf of normal distribution

def normal_dist(x , mean , sd):
    prob_density = (np.pi*sd) * np.exp(-0.5*((x-mean)/sd)**2)        #pdf of Normal distribution
    return prob_density



def operation(list_X, list_Y,beta):
  sumission=[0]*6  #initial 0
  for i in range(len(list_X)):
    a= (list_X[i])
    x= Matrix_multi(a,beta)          #to find value of transpose(xi)*beta
    if(norm.cdf(x) == 0):
      ans= list_Y[i]                    #edge cases occurs for very high or very low values of k in phi(k) so taking limits for them
    elif(norm.cdf(x) == 1):
      ans= list_Y[i]-1
    else:
      ans= ((list_Y[i]- norm.cdf(x))*(normal_dist(x,0,1)))/((1-norm.cdf(x))*(norm.cdf(x)))   #differentition of log likelihood 
    
    sumission= sumission+ ans*(list_X[i]) #adding each term of sumission 
  
  sumission= sumission.tolist()
  return sumission

  
def Matrix_multi(a,b):          ##to find value of transpose(xi)*beta
  sum=0
  for i in range(len(a)):
    sum= sum+ a[i]*b[i]
  return sum                                        

def difference(a,b):            #to calculate difference in next cell 
  sum=0
  for i in range(len(a)):
    sum= sum+ (a[i]-b[i])*(a[i]-b[i])
  return sum

In [ ]:
normal_dist(-20,0,1)

4.347639161724566e-87

In [ ]:
t= 0.00005
tolerance= 0.000045
iter=0
diff=100
beta= [1]*6
beta_new= beta
while (diff>tolerance and iter<110):
  iter= iter+1
  old= beta_new
  
  multiplied_list = [element * t for element in operation(list_X,list_Y,old)] #gradient ascent
  res_lt = [] # declaration of the list  
  for x in range (0, len (old)):  
    res_lt.append( old[x] + multiplied_list[x])  
  beta_new= res_lt                                     ## new beta_n
  
  diff= difference(beta_new,old)
  print(diff)
  

iter

In [ ]:
print("beta = ",beta_new)

list1= [1,1,0.246042,0,0,0.014639]
print("Probability of Jack = ",norm.cdf(Matrix_multi(list1,beta_new)))
list2= [1,0,0.233476,1,1,0.999357]
print("Probability of Rose = ",norm.cdf(Matrix_multi(list2,beta_new)))


beta =  [0.39927561828259867, -1.3416911416628843, 0.33549291620798843, -0.09768899754299433, 0.003458524667663765, 1.2296688206058102]
Probability of Jack =  0.19993062711360204
Probability of Rose =  0.9465465336883306
